In [ ]:
{
 "cells": [],
 "metadata": {},
 "nbformat": 4,
 "nbformat_minor": 2
}


{'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 2}

In [ ]:
pip install transformers datasets

In [ ]:
from google.colab import files
uploaded = files.upload()  # This lets you upload a file manually

import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer

# Load the uploaded CSV
df = pd.read_csv(next(iter(uploaded)))  # Gets the uploaded filename

# Remove rows where 'text' is missing
df = df.dropna(subset=['text'])

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Add padding token

# Tokenization function
def tokenize_function(example):
    tokens = tokenizer(example['text'], padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Saving sample_data.csv to sample_data.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import GPT2Tokenizer

# Read uploaded file
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Fix padding error

# Tokenize the input text
tokens = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True)

# Prepare dataset manually
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return self.input_ids.shape[0]

    def __getitem__(self, idx):
        return {"input_ids": self.input_ids[idx], "labels": self.input_ids[idx]}

# Split into chunks of max 1024 tokens
chunk_size = 512
input_ids_chunks = tokens['input_ids'].squeeze(0).split(chunk_size)
dataset = TextDataset(torch.stack(input_ids_chunks))


In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases logging

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
)
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("gpt2")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=3.165557861328125, metrics={'train_runtime': 28.1157, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.107, 'total_flos': 82674432000.0, 'train_loss': 3.165557861328125, 'epoch': 3.0})

In [ ]:
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=100, num_return_sequences=1)

print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Once upon a time, the world was a different place.


The world was a different place.

The world was a different place.

The world was a different place.

The world was a different place.
The world was a different place.

The world was a different place.

The world was a different place.

The world was a different place.

The world was a different place.

The world was a different


In [ ]:
model.save_pretrained("my-finetuned-gpt2")
tokenizer.save_pretrained("my-finetuned-gpt2")


('my-finetuned-gpt2/tokenizer_config.json',
 'my-finetuned-gpt2/special_tokens_map.json',
 'my-finetuned-gpt2/vocab.json',
 'my-finetuned-gpt2/merges.txt',
 'my-finetuned-gpt2/added_tokens.json')

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load your fine-tuned model
model = GPT2LMHeadModel.from_pretrained("my-finetuned-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("my-finetuned-gpt2")

# Provide a prompt
prompt = "Once upon a time"
input_ids = tokenizer(prompt, return_tensors='pt').input_ids

# Generate text
output = model.generate(input_ids, max_length=100, num_return_sequences=1)
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger.

The world was a place of great danger, and the world was a place of great danger.

The world was a place of great danger, and the world was a place of great danger.

The world was a place of great danger, and the world was a place of great danger.



In [ ]:
from google.colab import files
uploaded = files.upload()
